In [ ]:
!pip install requests beautifulsoup4
!pip install firebase
import re
from nltk.stem import PorterStemmer

class IndexService:
    def __init__(self):
        self.index = {}

    def index_words(self, text):
        """יוצר אינדקס של מילים מתוך טקסט"""
        words = re.findall(r'\w+', text)
        for word in words:
            word = word.lower()
            if word in self.index:
                self.index[word] += 1
            else:
                self.index[word] = 1

    def remove_stop_words(self):
        """מסיר מילים נפוצות מהאינדקס"""
        stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at'}
        for stop_word in stop_words:
            self.index.pop(stop_word, None)

    def apply_stemming(self):
        """ממיר מילים לשורשים שלהן (Stemming)"""
        stemmer = PorterStemmer()
        stemmed_index = {}
        for word, count in self.index.items():
            stemmed_word = stemmer.stem(word)
            if stemmed_word in stemmed_index:
                stemmed_index[stemmed_word] += count
            else:
                stemmed_index[stemmed_word] = count
        self.index = stemmed_index

    def search(self, query):
        """מחפש מילות מפתח באינדקס"""
        stemmer = PorterStemmer()
        query_words = re.findall(r'\w+', query.lower())
        results = {}
        for word in query_words:
            word = stemmer.stem(word)
            if word in self.index:
                results[word] = self.index[word]
        return results

    def get_index(self):
        """מחזיר את האינדקס הנוכחי"""
        return self.index


In [20]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

def fetch_page(url):
    try:
        response = urlopen(url)
        return BeautifulSoup(response, 'html.parser')
    except:
        return None

In [21]:
from firebase import firebase

class FirebaseService:
    def __init__(self, db_link: str):
        self.conn = firebase.FirebaseApplication(db_link, None)

    def put(self, path: str, data: dict):
        """מכניס נתונים ל-Firebase תחת הנתיב שצוין"""
        self.conn.put('/', path, data)
        print(f"✅ הנתונים נשמרו בהצלחה תחת /{path}")

    def fetch(self, path: str):
        """שולף נתונים מ-Firebase מתוך הנתיב שצוין"""
        result = self.conn.get(f"/{path}", None)
        print(f"📥 נתונים התקבלו מתוך /{path}")
        return result


In [ ]:
# טען את הדף מהאינטרנט
url = 'https://www.arianagrande.com/'
soup = fetch_page(url)
fire = FirebaseService("https://cloud7-38a0b-default-rtdb.firebaseio.com/")
# צור אינדקס, נקה מילים נפוצות, והחזר תוצאות חיפוש
if soup:
    service = IndexService()
    service.index_words(soup.get_text())      # שלב 1: אינדוקס
    service.remove_stop_words()               # שלב 2: הסרת מילים נפוצות
    service.apply_stemming()                  # שלב 3: קיצור למילים בסיסיות (stemming)
    result = service.search('shop buy ariana music grande video eternal sunshine brighter days')

    print(result)
    fire.put("words",result)
    print(fire.fetch("words"))
    query = "eternal"
    results = service.search(query)
    print("\n🔎 תוצאות חיפוש עבור השאילתה:", query)
    print(results)
else:
    print("לא ניתן לטעון את הדף.")
